In [313]:
import numpy as np
import os
import pandas as pd
from pathlib import Path

__file__ = Path(os.path.abspath('__file__'))
root_path = __file__.parent
data_path = root_path / 'data'

sets = ['train', 'validation', 'test', 'all']

print(f'{__file__  = }')

__file__  = PosixPath('/home/shane/workspace/hackathon-ddh/__file__')


In [314]:
dfs = {}
for s in sets:
    dfs[s] = pd.read_csv(f'new/{s}.csv')

In [315]:
dfs['all'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10284 entries, 0 to 10283
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   person_id      10284 non-null  int64  
 1   type           10284 non-null  object 
 2   file_name      10284 non-null  object 
 3   direction      10284 non-null  object 
 4   since          10284 non-null  object 
 5   sex            10284 non-null  object 
 6   age            10284 non-null  int64  
 7   date           10284 non-null  int64  
 8   detailScore    10284 non-null  object 
 9   totalScore     10284 non-null  int64  
 10  segmentation   10240 non-null  object 
 11  keypoints      10281 non-null  object 
 12  num_keypoints  10284 non-null  int64  
 13  containment    7685 non-null   float64
 14  diagnose       7685 non-null   object 
 15  a_angle        10044 non-null  float64
 16  a_diagnose     10270 non-null  object 
 17  b_angle        9214 non-null   float64
 18  b_diag

In [316]:
columns = [
    'sex',              # 'm' or 'f' no nan  -> male: 1, female: 0
    'age',              # int [0, 15] no nan
    'detailScore',      # parsing needed == 5
    'totalScore',       # int no nan
    'keypoints',        # parsing needed
    'containment',      # int.0 positive with nan
    'diagnose',         # {'normal': 6509, nan: 2599, 'abnormal': 1176}
    'a_angle',          # int.0 or nan
    'a_diagnose',       # {'normal': 7458, 'mild Dysplasia': 2465, 'severe dysplasia': 237, 'moderate dysplasia': 110, nan: 14} 0 1 3 2 None
    'b_angle',          # int.0 or nan
    'b_diagnose',       # {'normal': 8568, 'severe dysplasia': 1058, 'mild Dysplasia': 633, 'moderate dysplasia': 12, nan: 13} 0 3 1 2 None
    'alpha',            # int' 'int parsing needed
    'alpha_angle',      # int.0 or nan
    'alpha_ratio',      # positive float [0, inf)
    'usage',            # train, validation, test
]

In [317]:
df = dfs['all'][columns].copy()
df = df.drop(['detailScore', 'keypoints', 'alpha'], axis=1)
df.shape

(10284, 12)

In [318]:
detail_score = np.array(list(map(lambda x: list(map(int, x.split(' '))) ,dfs['all']['detailScore'])))
for i in range(5):
    df[f'detail_score_{i}'] = detail_score[:, i:i+1]


keypoints = list(map(lambda x: list(map(lambda i: int(i) if i.isnumeric() else None, x.split(' '))), map(str, dfs['all']['keypoints'])))
for i in range(len(keypoints)):
    if len(keypoints[i]) != 18:
        keypoints[i] = [None] * 18

keypoints = np.array(keypoints)
for i in range(18):
    df[f'keypoints_{i}'] = keypoints[:, i:i+1]

alpha = list(map(lambda x: list(map(lambda i: int(i) if i.isnumeric() else None, x.split(' '))), map(str, dfs['all']['alpha'])))
for i in range(len(alpha)):
    if len(alpha[i]) != 2:
        alpha[i] = [None] * 2

alpha = np.array(alpha)
for i, c in enumerate('xy'):
    df[f'alpha_{c}'] = alpha[:, i:i+1]

In [319]:
df # 10283
df = df.dropna() # 7152

In [320]:
df['sex'][df['sex'] == 'm'] = 1
df['sex'][df['sex'] == 'f'] = 0

# ['a_diagnose', 'b_diagnose']
dig_1 = {
    'normal': 0,
    'mild Dysplasia': 1,
    'moderate dysplasia': 2,
    'severe dysplasia': 3,
}

# ['diagnose']
dig_2 = {
    'normal': 0,
    'abnormal': 1,
}

for col in ['a_diagnose', 'b_diagnose']:
    for k, v in dig_1.items():
        df[col][df[col] == k] = v

for col in ['diagnose']:
    for k, v in dig_2.items():
        df[col][df[col] == k] = v

/tmp/ipykernel_47617/8636922.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'][df['sex'] == 'f'] = 0
/tmp/ipykernel_47617/8636922.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df[col] == k] = v
/tmp/ipykernel_47617/8636922.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df[col] == k] = v


In [321]:
df.to_csv('data.csv', index=False)